In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta # Importação necessária

artista = "3Palavrinhas"



df_total = pd.read_csv('dados_full/3Palavrinhas/total.csv')
if not df_total.empty:
    df_total = df_total.drop(df_total.index[0])
df_total['Data'] = pd.to_datetime(df_total['Data'], format='%Y-%m', errors='coerce')
df_total.dropna(subset=['Data'], inplace=True)
ultima_data = df_total['Data'].max()


# --- NOVO BLOCO DE CÓDIGO APLICADO ---
meses_para_analise = []
data_referencia = ultima_data # A data mais recente do arquivo total.csv

# Loop de 5 meses para trás até o mês de referência (delta 0)
for i in range(5, -1, -1):
    mes_alvo = data_referencia - relativedelta(months=i)
    meses_para_analise.append(mes_alvo.strftime('%Y-%m'))
# --- FIM DO NOVO BLOCO ---


coluna_data_publicacao = 'Horário de publicação do vídeo'

dfs=[]
for i, mes_str_analise in enumerate(meses_para_analise):
    nome_arquivo_mes = str(i + 1).zfill(2)

    caminho = f'dados_full/3Palavrinhas/videos_{nome_arquivo_mes}.csv'
    df_mes = pd.read_csv(caminho)

    if coluna_data_publicacao in df_mes.columns:
        datas_convertidas = pd.to_datetime(df_mes[coluna_data_publicacao], format='%b %d, %Y', errors='coerce')
        df_mes['mes_publicacao'] = datas_convertidas.dt.strftime('%Y-%m')
        df_mes = df_mes[df_mes['mes_publicacao'] == mes_str_analise]
        dfs.append(df_mes)

# Agora a lista 'dfs' conterá os DataFrames filtrados para os meses de Março a Agosto (inclusive).

df_concatenado = pd.concat(dfs, ignore_index=True)

if 'Duração média da visualização' in df_concatenado.columns:
    # Converte para segundos
    def duracao_para_segundos(x):
        if pd.isnull(x):
            return None
        try:
            partes = str(x).split(':')
            partes = [int(p) for p in partes]
            if len(partes) == 3:
                h, m, s = partes
            elif len(partes) == 2:
                h = 0
                m, s = partes
            elif len(partes) == 1:
                h = 0
                m = 0
                s = partes[0]
            else:
                return None
            return h*3600 + m*60 + s
        except Exception:
            return None

    df_concatenado['Duração média da visualização'] = df_concatenado['Duração média da visualização'].apply(duracao_para_segundos)

df_concatenado.head(5)


df_agrupado = (df_concatenado
               .groupby('mes_publicacao', as_index=False)
               .agg({
                   'Duração':'mean',
                   'Receita estimada (USD)':'sum',
                   'Porcentagem visualizada média (%)':'mean',
                   'Impressões':'sum',
                   'Taxa de cliques de impressões (%)':'mean',
                   'RPM (USD)':'mean',
                   'Marcações "Gostei"':'sum',
                   'Compartilhamentos':'sum',
                   'Comentários adicionados':'sum',
                   'Espectadores únicos':'sum',
                   'Visualizações':'sum',
                   'Tempo de exibição (horas)':'sum',
                   'Duração média da visualização':'mean'
               }))

df_agrupado

,mes_publicacao,Duração,Receita estimada (USD),Porcentagem visualizada média (%),Impressões,Taxa de cliques de impressões (%),RPM (USD),"Marcações ""Gostei""",Compartilhamentos,Comentários adicionados,Espectadores únicos,Visualizações,Tempo de exibição (horas),Duração média da visualização
0,2025-03,167.250000,1000.931,66.862500,9675728.0,5.997500,0.417000,5063.0,1088.0,0.0,1058649.0,2726759.0,96672.9073,110.500000
1,2025-04,180.000000,2876.506,67.628000,25901010.0,6.772000,0.374000,22082.0,15243.0,0.0,2790104.0,7702141.0,312904.2351,122.200000
2,2025-05,188.166667,2970.043,70.838333,20813831.0,5.930000,0.463167,12518.0,2898.0,0.0,2513531.0,6474817.0,278554.3828,135.333333
3,2025-06,201.500000,1562.723,62.967500,11011118.0,5.872500,0.423500,5449.0,723.0,0.0,1370428.0,3647634.0,145996.0821,124.250000
4,2025-07,277.833333,849.050,42.073333,12382321.0,5.061667,0.466833,6965.0,1259.0,0.0,917299.0,1742121.0,50683.4035,94.666667
5,2025-08,176.800000,2315.913,69.338000,18631728.0,6.232000,0.471000,15046.0,14161.0,0.0,1912648.0,4710521.0,190005.1624,122.200000
